In [1]:
import duckdb

In [4]:
conn = duckdb.connect()

In [5]:
duckdb.sql(
    """SELECT *
           FROM read_json_auto('Data/*/*.json') LIMIT 1"""
           )



┌─────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Defining User Defined Functions
This section is for defining user defined functions that allows run custom functions on the dataset. 

In [71]:
import re 
import requests
from selectolax.parser import HTMLParser

def extract_opening_data(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/115.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Referer": "https://www.google.com",
    }

    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            tree = HTMLParser(response.text)
            
            # Extract H1 (Opening Name)
            h1_tag = tree.css_first("h1")
            opening_name = h1_tag.text(strip=True) if h1_tag else None
            
            # Extract Moves List
            move_list_div = tree.css_first(".openings-view-move-list")
            moves = move_list_div.text(strip=True) if move_list_div else None

            return {"opening_name": opening_name, "moves": moves}

    except requests.RequestException:
        return {"opening_name": None, "moves": None}  # Handle errors

    return {"opening_name": None, "moves": None}

# Define Function to add move numbers to pgn
def add_move_numbers(pgn: list) -> str:
    # Remove any existing move numbers and split into individual moves
    moves = pgn
    
    # Reconstruct the PGN with move numbers
    formatted_pgn = []
    move_number = 1
    for i in range(0, len(pgn), 2):
        # Add the move number and the two moves (white and black)
        formatted_pgn.append(f"{move_number}. {moves[i]} {moves[i+1] if i+1 < len(moves) else ''}")
        move_number += 1
    
    # Join the formatted moves into a single string
    return ' '.join(formatted_pgn)

# duckdb.remove_function('add_move_numbers')
# duckdb.create_function('add_move_numbers', add_move_numbers)


def get_opening_family(opening_name: str) -> str:
    # Get the parent name of the move numbers

    if ":" in opening_name:
        name_splitted = opening_name.split(":")
        return name_splitted[0]

    else:
        return opening_name
    
# duckdb.create_function('get_opening_family', get_opening_family)

def get_pgn_depth(pgn: str) -> int:
    # this function is supposed to define the number of moves from a pgn file. 
    moves = re.findall(r'\d+\.', pgn)  # Find all move numbers (e.g., '1.', '2.', etc.)
    return int(len(moves)) # Return the number of moves

# duckdb.create_function('get_pgn_depth', get_pgn_depth)


def extract_opening_name(url: str) -> str:
    # extract Opening Data
    opening_data = extract_opening_data(url)
    return opening_data['opening_name']

duckdb.create_function('extract_opening_name', extract_opening_name) 
    

## Performing Transformation on Json Data For Fact Table. 

In [7]:
import pandas as pd

In [9]:
fact_table = duckdb.sql(
    """SELECT url as game_url,
                time_control as time_control,
                rated as rated,
                time_class as time_class,
                rules as rules,
                white.rating as white_rating,
                white.result as white_result,
                black.rating as black_rating,
                black.result as black_result,    
                REGEXP_EXTRACT(pgn, '\[Event "(.*?)"', 1) as pgn_event,
                REGEXP_EXTRACT(pgn, '\[Site "(.*?)"', 1) as pgn_site, 
                STRPTIME(REPLACE(REGEXP_EXTRACT(pgn, '\[Date "(.*?)"', 1), '.', '/'), '%Y/%m/%d')::DATE AS game_date,
                REGEXP_EXTRACT(pgn, '\[White "(.*?)"', 1) as pgn_white_user,   
                REGEXP_EXTRACT(pgn, '\[Black "(.*?)"', 1) as pgn_black_user,
                REGEXP_EXTRACT(pgn, '\[Result "(.*?)"', 1) as pgn_result,
                REGEXP_EXTRACT(pgn, '\[CurrentPosition "(.*?)"', 1) as pgn_current_position,
                REGEXP_EXTRACT(pgn, '\[Timezone "(.*?)"', 1) as pgn_timezone,
                REGEXP_EXTRACT(pgn, '\[ECO "(.*?)"', 1) as pgn_eco,
                REGEXP_EXTRACT(pgn, '\[ECOUrl "(.*?)"', 1) as pgn_eco_url,
                STRPTIME(REGEXP_EXTRACT(pgn, '\[StartTime "(.*?)"', 1), '%H:%M:%S'):: TIME as start_time,
                STRPTIME(REGEXP_EXTRACT(pgn, '\[EndTime "(.*?)"', 1), '%H:%M:%S'):: TIME as end_time,
                STRPTIME(REPLACE(REGEXP_EXTRACT(pgn, '\[EndDate "(.*?)"', 1), '.', '/'), '%Y/%m/%d')::DATE AS end_game_date,
                ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(pgn, '\. (.*?) {\[', 1), ' ') as pgn_raw,
                add_move_numbers(REGEXP_EXTRACT_ALL(pgn, '\. (.*?) {\[', 1)) as pgn_trans
                                
            FROM read_json_auto('Data/*/*.json')"""
           )
fact_table

┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬─────────────────┬────────────────┬────────────┬───────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
fact_5 = duckdb.sql(
    """SELECT * 
           FROM fact_table LIMIT 5"""
           )
duckdb.sql('SELECT DISTINCT(pgn_eco_url) FROM fact_table')

┌───────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                            pgn_eco_url                                            │
│                                              varchar                                              │
├───────────────────────────────────────────────────────────────────────────────────────────────────┤
│ https://www.chess.com/openings/Ruy-Lopez-Opening-Cozio-Defense-4.O-O-g6                           │
│ https://www.chess.com/openings/Sicilian-Defense-McDonnell-Attack-2...e6-3.Nf3-d5                  │
│ https://www.chess.com/openings/Ruy-Lopez-Opening                                                  │
│ https://www.chess.com/openings/Sicilian-Defense-Open...4.Nxd4-d6-5.Nc3-g6                         │
│ https://www.chess.com/openings/Englund-Gambit-Declined                                            │
│ https://www.chess.com/openings/French-Defense-Exchange-Monte-Carlo-Variation    

In [11]:
fct = fact_table.df()

fct['start_time'] = pd.to_datetime(fct['game_date'].astype(str) +  " " + fct['start_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
fct['end_time'] = pd.to_datetime(fct['end_game_date'].astype(str) + " " + fct['end_time'].astype(str), format='%Y-%m-%d %H:%M:%S')


In [ ]:
# # Transform openings csv 
# openings = conn.sql("""
#             SELECT "eco-volume" as eco_family, eco, name, pgn
#             FROM 'hf://datasets/Lichess/chess-openings/data/train-00000-of-00001.parquet'
            
#          """
# ).df()
# openings

openings = duckdb.sql("""
    SELECT *, get_opening_family(name) as family_name
           FROM read_csv('openings.csv')
           
""")
openings

In [ ]:
# Create Dimensional Tabless
duckdb.sql("""SELECT DISTINCT 
       CASE 
           WHEN pgn_result = '1-0' THEN 'Win'
           WHEN pgn_result = '0-1' THEN 'Loss'
           WHEN pgn_result = '1/2-1/2' THEN 'Draw'
           ELSE 'Other'
       END AS result_category,
       white_result AS result_detail
FROM fct;
""")

# 
duckdb.sql("""SELECT DISTINCT time_control,
       CASE 
           WHEN time_control LIKE '600%' THEN 'Rapid'
           WHEN time_control LIKE '300%' OR time_control LIKE '180%' THEN 'Blitz'
           WHEN time_control LIKE '120%' OR time_control LIKE '60%' THEN 'Bullet'
           ELSE 'Other'
       END AS time_class
FROM fct;
""")

dim_openings = duckdb.sql("""
        SELECT DISTINCT pgn_eco_url, pgn_eco, extract_opening_name(pgn_eco_url) as opening_name
           FROM fct 
""")
dim_openings

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┬──────────────────────────────────────────────────────────────┐
│                                              pgn_eco_url                                               │ pgn_eco │                         opening_name                         │
│                                                varchar                                                 │ varchar │                           varchar                            │
├────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┼──────────────────────────────────────────────────────────────┤
│ https://www.chess.com/openings/French-Defense-Knight-Variation-2...d5                                  │ C00     │ French Defense: Knight Variation                             │
│ https://www.chess.com/openings/Sicilian-Defense-McDonnell-Attack-2...e6-3.Nf3-d5                  

In [76]:
fct_final = duckdb.sql("""
            SELECT game_url as game_url,
                       game_date as game_date,
                       start_time as start_time,
                       end_time as end_time,
                       time_control, 
                       CASE WHEN pgn_white_user = 'Rhythmbear1' THEN 'white'
                            ELSE 'black' END as my_color,
                       CASE 
                            WHEN pgn_white_user = 'Rhythmbear1' THEN white_rating
                            ELSE black_rating END as my_rating,
                       CASE 
                            WHEN pgn_white_user = 'Rhythmbear1' THEN black_rating
                            ELSE white_rating END as opponent_rating,
                       CASE 
                            WHEN pgn_white_user = 'Rhythmbear1' THEN white_result
                            ELSE black_result END as my_result,
                       CASE 
                            WHEN pgn_white_user = 'Rhythmbear1' THEN black_result
                            ELSE white_result END as opponent_result,
                       pgn_eco_url as opening_url,
                       pgn_trans
                       
                       FROM fct
                       
                       """
    
)
fct_final

┌──────────────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬──────────────┬──────────┬───────────┬─────────────────┬────────────┬─────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Download Openings from Li-chess database.

In [12]:
duckdb.query("""SELECT *
                FROM fact_table as ft
                WHERE pgn_trans LIKE '%1. e4 e6%'         
             """)

┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬─────────────────┬────────────────┬────────────┬─────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────

In [13]:
duckdb.sql("""
            SELECT family_name, count(*) as num_opens FROM openings GROUP BY family_name ORDER BY num_opens desc;
        """)

┌───────────────────────────────────┬───────────┐
│            family_name            │ num_opens │
│              varchar              │   int64   │
├───────────────────────────────────┼───────────┤
│ Sicilian Defense                  │       371 │
│ Ruy Lopez                         │       229 │
│ French Defense                    │       188 │
│ Italian Game                      │       169 │
│ Queen's Gambit Declined           │       163 │
│ English Opening                   │       146 │
│ King's Gambit Accepted            │       137 │
│ King's Indian Defense             │       112 │
│ Caro-Kann Defense                 │       103 │
│ Nimzo-Indian Defense              │        92 │
│          ·                        │         · │
│          ·                        │         · │
│          ·                        │         · │
│ Pseudo Queen's Indian Defense     │         1 │
│ Barnes Defense                    │         1 │
│ Czech Defense                     │         1 │


In [14]:
# Create Dimension tables for Openings, Date, Player and GameType

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS dim_openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")


BinderException: Binder Error: Referenced column "pgn_eco" not found in FROM clause!
Candidate bindings: "read_json_auto.pgn", "read_json_auto.end_time", "read_json_auto.time_control", "read_json_auto.initial_setup"
LINE 4:     pgn_eco AS opening_code,
            ^

In [8]:
duckdb.sql("""  
                SELECT DISTINCT(opening_family) FROM read_parquet('https://rbchesssa.blob.core.windows.net/chess-etl-files/gold/dim_openings.parquet')
        """)  

┌──────────────────────────────┐
│        opening_family        │
│           varchar            │
├──────────────────────────────┤
│ Englund Gambit Declined      │
│ Bird's Opening               │
│ English Opening              │
│ Nimzowitsch Defense Declined │
│ French Defense               │
│ Modern Defense with 1.e4     │
│ Center Game Accepted         │
│ Englund Gambit               │
│ Queen's Gambit Declined      │
│ Caro-Kann Defense            │
│         ·                    │
│         ·                    │
│         ·                    │
│ King's Gambit Accepted       │
│ Alapin Sicilian Defense      │
│ Queen's Gambit Accepted      │
│ Sicilian Defense             │
│ Réti Opening                 │
│ Bishop's Opening             │
│ Owen's Defense               │
│ Closed Sicilian Defense      │
│ The Cow                      │
│ King's Fianchetto Opening    │
├──────────────────────────────┤
│      43 rows (20 shown)      │
└──────────────────────────────┘

In [ ]:
! pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 3.5 MB/s eta 0:00:0000:0100:01m


In [ ]:
duckdb.sql(""" SELECT * FROM read_parquet('https://rbchesssa.blob.core.windows.net/chess-etl-files/silver/fact-2025-03-games.parquet') """)


┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬─────────────────────┬────────────────────┬──────────────────┬────────────┬──────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Creating the Function to read from the Postgres database.


In [15]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

db_name = os.getenv("az_dw_name")
user = os.getenv("az_dw_username")
password = os.getenv("az_dw_password")
host = os.getenv("az_dw_host")


conn = duckdb.connect(":memory:")

conn.sql("""
               INSTALL POSTGRES;
               LOAD POSTGRES; 
             """)


In [16]:

conn.sql(f"""ATTACH 'dbname={db_name} user={user} password={password} host={host} port=5432' 
         AS chess_db (TYPE postgres, SCHEMA 'chess_dw') ;""")

IOException: IO Error: Unable to connect to Postgres at dbname=postgres user=rhythmbear password=Okechukwu10. host=chess-dw.postgres.database.azure.com port=5432: could not translate host name "chess-dw.postgres.database.azure.com" to address: Temporary failure in name resolution


In [ ]:
conn.sql(f""" SELECT * FROM chess_db.fact_games LIMIT 10; """)

┌──────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬───────────┬──────────┬───────────┬────────────────┬────────────────┬────────────┬──────────────────────┬──────────────┬─────────┬─────────────┬────────────┬───────────┬───────────────┬─────────┬───────────┬──────────────┐
│ game_url │ time_control │  rated  │ time_class │  rules  │ white_rating │ white_result │ black_rating │ black_result │ pgn_event │ pgn_site │ game_date │ pgn_white_user │ pgn_black_user │ pgn_result │ pgn_current_position │ pgn_timezone │ pgn_eco │ pgn_eco_url │ start_time │ end_time  │ end_game_date │ pgn_raw │ pgn_trans │ last_updated │
│  int32   │   varchar    │ boolean │  varchar   │ varchar │    int32     │   varchar    │    int32     │   varchar    │  varchar  │ varchar  │   date    │    varchar     │    varchar     │  varchar   │       varchar        │   varchar    │ varchar │   varchar   │ timestamp  │ timestamp │   timestamp   │ varchar 

## Get Opening from the dimension table. 
The goal here is to define what opening was played for each game by searching through tht eopenings database from lichess and getting the opening that best describes the game. 

In [20]:
# Defint the function to search through
# def extract_opening(pgn):
    # Load the opening dimension table into a variable. 
openings = duckdb.sql(""" SELECT *, get_pgn_depth(pgn) as moves_count FROM read_csv('openings.csv') """)
print(duckdb.sql(""" SELECT COUNT(DISTINCT(name)), COUNT(DISTINCT(pgn)), COUNT(*), COUNT(DISTINCT(name, eco)) FROM read_csv('openings.csv') """))
openings

┌────────────────────────┬─────────────────────┬──────────────┬─────────────────────────────────────────┐
│ count(DISTINCT "name") │ count(DISTINCT pgn) │ count_star() │ count(DISTINCT main."row"("name", eco)) │
│         int64          │        int64        │    int64     │                  int64                  │
├────────────────────────┼─────────────────────┼──────────────┼─────────────────────────────────────────┤
│                   3107 │                3513 │         3513 │                                    3256 │
└────────────────────────┴─────────────────────┴──────────────┴─────────────────────────────────────────┘



┌────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────┐
│ eco_family │   eco   │                                      name                                       │                                                                      pgn                                                                       │ moves_count │
│  varchar   │ varchar │                                     varchar                                     │                                                                    varchar                                                                     │    int64    │
├────────────┼─────────┼─────────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────

In [21]:


sample_pgn = "1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. O-O fxg3 6. hxg3"

def generate_pgn_dict(sample_pgn: str) -> dict:
    """ The Goal of this Function is to convert the pgn into a python dictionary that can be easier to work with."""
    # Split the moves into a list.
    moves = sample_pgn.split(" ")
    pgn_dict = {}

    for i in range(0, len(moves), 3):
        key = moves[i].strip(".")
        # Loop through the moves and add the moves to the dictionary checking to ensure that the index does not exceed the length of the moves.
        pgn_dict[int(key)] = [moves[i+1] if i+1 < len(moves) else '', moves[i+2] if i+2 < len(moves) else ''] 
    return pgn_dict
print(generate_pgn_dict(sample_pgn))

{1: ['Nh3', 'd5'], 2: ['g3', 'e5'], 3: ['f4', 'Bxh3'], 4: ['Bxh3', 'exf4'], 5: ['O-O', 'fxg3'], 6: ['hxg3', '']}


In [ ]:
# Extracting the openings from the pgn
test_pgn = "1. e4 e6 2. Nf3 d5 3. exd5 exd5 4. Nc3 c6 5. g3 Bd6 6. Bg2 Ne7 7. O-O O-O 8. d3 Bg4 9. Bg5 f6 10. Bh4 Nd7"
test_pgn_2 = "1. e4 d5 2. exd5 Qxd5 3. Nc3 Qd8 4. d4 Nf6 5. Bf4 e6 6. Bd3 Nc6 7. Nf3 Bb4 8. O-O b6 9. Re1 Bxc3 10. bxc3 Bb7"
test_pgn_3 = "1. e4 e6 2. Nf3 d5 3. exd5 exd5 4. d4 Bd6 5. Nc3 Nf6 6. Bg5 c6 7. Ne5 O-O 8. Be2 Re8 9. Nf3 h6"
test_pgn_4 = "1. e4 b6 2. Nf3 Bb7 3. Nc3 g6 4. d4 Bg7 5. Bd3 Na6 6. O-O Nb4 7. e5 Nxd3 8. Qxd3 e6 9. Re1 Qe7 10. Bg5 f6"


def get_opening(pgn: str) -> str:
    pgn_dic = generate_pgn_dict(pgn)

    prev_match = None

    pgn_flag = ""
    for move in pgn_dic:
        # print(move)
        pgn_flag += f"{move}." if move == 1 else f" {move}."
        for u_mv in pgn_dic[move]:
            pgn_flag += f" {u_mv}"
            # print(pgn_flag)
        
            matches = duckdb.sql(f""" SELECT *, get_opening_family(name) as family_name FROM openings where pgn LIKE '{pgn_flag}%' ORDER BY moves_count """)
            num_of_results = len(matches.df()['name'].to_list())
            # print(f"Number of results : {num_of_results}")
            if num_of_results == 0:
                # print(matches)
                break
            else:
                prev_match = matches 
                
        if num_of_results == 0:
            # print(matches)
            break  

        pgn_flag = pgn_flag.strip(" ")
    # print(prev_match)
    return prev_match.fetchone()[-1]  
# duckdb.create_function('get_opening', get_opening) 

NotImplementedException: Not implemented Error: A function by the name of 'get_opening' is already created, creating multiple functions with the same name is not supported yet, please remove it first

## Scrape Chess.com to extract more details about the opening url.

In [53]:
! pip install selectolax

In [ ]:
res = extract_opening_data("https://www.chess.com/openings/Ruy-Lopez-Opening-Cozio-Defense-4.O-O-g6")
print(res)

{'opening_name': 'Ruy López Opening: Cozio Defense', 'moves': '1.e4 e5 2.Nf3 Nc6 3.Bb5 Nge7 4.O-O g6'}
